In [20]:
import openai
from langchain.embeddings.base import Embeddings
from pydantic import Field, BaseModel
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate

### 文档切分

In [3]:
# 读取原始文档
FILE_PATH = "/data01/tqbian/src/learning/rag_retriever/人物介绍.txt"
# with open(FILE_PATH, 'r', encoding='utf-8') as f:
#     documents = f.read()
loader = TextLoader(FILE_PATH)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=100, separators=["。", "？", "！"], add_start_index=True)
# 切分文本
texts = text_splitter.split_documents(documents)
# for text in texts:
#     print(text)
print(len(texts))
print(len(texts[0].page_content))
print(texts[10].metadata)

92
206
{'source': '/data01/tqbian/src/learning/rag_retriever/人物介绍.txt', 'start_index': 1534}


### 自定义embedding模型

In [21]:
_params = {
        "base_url": 'http://127.0.0.1:9997/v1',
        "api_key": 'EMPTY',
    }
# 自定义嵌入类
class XinferenceOpenAIEmbeddings(BaseModel, Embeddings):
    model: str = Field(default='m3e-base', description="使用的嵌入模型名称")
    client: object = Field(default=None, description="")        
    def embed_documents(self, texts):
        response = self.client.embeddings.create(
            model=self.model,
            input=texts
        )
        embeddings = [data.embedding for data in response.data]
        return embeddings

    def embed_query(self, text):
        response = self.client.embeddings.create(
            model=self.model,
            input=[text]
        )
        embeddings = response.data[0].embedding
        return embeddings
# 初始化嵌入实例
client = openai.Client(**_params)
embeddings = XinferenceOpenAIEmbeddings(model='m3e-base', client=client)

### 测试embedding模型

In [5]:
# 示例文档
documents = [
    "这是第一个文档的内容。",
    "这是第二个文档的内容。",
    "第三个文档也包含一些有用的信息。"
]
doc_embeddings = embeddings.embed_documents(documents)
print("文档嵌入向量:")
for i, emb in enumerate(doc_embeddings):
    print(f"文档 {i+1} 嵌入向量长度: {len(emb)}")
query = "什么是检索增强生成（RAG）？"
query_embedding = embeddings.embed_query(query)
print("\n查询嵌入向量:")
print(f"查询嵌入向量长度: {len(query_embedding)}")

文档嵌入向量:
文档 1 嵌入向量长度: 768
文档 2 嵌入向量长度: 768
文档 3 嵌入向量长度: 768

查询嵌入向量:
查询嵌入向量长度: 768


### 定义LLM

In [10]:
params = {
        "model_name" : "chatglm3",
        "temperature": 0.7,
        "base_url": 'http://127.0.0.1:9997/v1',
        "api_key": 'EMPTY',
    }
llm = ChatOpenAI(**params)

### 测试LLM

In [11]:
query = "北京奥运会"
messages = [
    (
        "system",
        "你是一个智能助手，可以回答任何问题",
    ),
    ("human", f"{query}"),
]
llm.invoke(messages)

AIMessage(content='北京奥运会，即2008年北京奥运会，是第29届夏季奥林匹克运动会，于2008年8月8日至24日在中华人民共和国北京市举行。这是中国首次举办奥运会，也使北京成为世界上第一个既举办过夏季奥运会又举办过冬季奥运会的城市。\n\n北京奥运会共有204个国家和地区以及俄罗斯、乌克兰等两个地区性的组织参赛，共有2,920名运动员参加了这次比赛。在这次比赛中，中国代表团表现出色，共获得了51枚金牌，成为金牌榜的领先者。\n\n北京奥运会的举办，不仅提升了我国的国际地位，展示了我国综合国力和举办大型国际活动的的能力，也为我国体育事业的发展带来了新的机遇。同时，北京奥运会也推动了我国城市基础设施建设，特别是交通、环保等方面的进步。', response_metadata={'token_usage': {'completion_tokens': -1, 'prompt_tokens': -1, 'total_tokens': -1}, 'model_name': 'chatglm3', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b142af25-0e0b-49dd-b04d-f980bee925fc-0')

### 创建向量数据库，创建检索
使用chroma数据库
- search_kwargs：为top k的值

In [13]:
chroma_db = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
    )
# 创建检索器
retriever = chroma_db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

### 测试检索

In [14]:
retriever_docs = retriever.invoke("姚明妻子是是谁")
len(retriever_docs)

6

In [15]:
for res in retriever_docs:
    print(res.page_content)

。通过一部系列电视片，他筹得30万美元用以帮助抵抗疾病。其妻叶莉，前中国女篮队员，姚明17岁就与她相识。2007年8月3日，姚明和叶莉在上海徐汇区婚姻登记处正式领取结婚证书，2007年8月6日，两人举办婚礼[74]。其女姚沁蕾（Amy），2010年5月22日在美国休斯敦出生[75]，取名Amy[76][77][78]，拥有美国国籍[79]，在美国长大，为美国公民。姚明曾表示等他女儿成年，再自行决定是否要申请中华人民共和国国籍，放弃美国国籍
。姚明是中国最具影响力的人物之一，同时也是世界最知名的华人运动员之一[4][5]。2009年，姚明收购上海男篮，成为上海大鲨鱼篮球俱乐部老板[6]。2011年7月20日，姚明正式宣布退役[7][8]。2016年11月22日，姚明出任CBA联盟副董事长。2017年2月，姚明当选为中国篮球协会主席[9]。2016年4月4日，姚明与前NBA球星沙奎尔·奥尼尔和艾伦·艾弗森一同入选奈史密斯篮球名人纪念堂，他也是首位入选也是迄今为止唯一入选名人堂的亚洲球员
。姚明曾表示等他女儿成年，再自行决定是否要申请中华人民共和国国籍，放弃美国国籍。叶莉（1981年11月20日—）是上海人，中国女子篮球运动员，曾经效力上海东方八爪鱼队，位置是中锋，她身高1.93米。叶莉1996年进入上海体育运动技术学院，1998年入选国家青年队，1999年底进国家集训队。生涯最好的成绩是2004年代表中国国家女子篮球队夺得亚洲锦标赛冠军。


老子（前571年—前470年），李氏，一说姓老，一说姓利，名耳或贞，字聃。一称老聃，一字伯阳
姚明（1980年9月12日—），男，出生于上海市徐汇区，祖籍江苏吴江。中国篮球运动员，曾为中国国家篮球队队员，曾效力于中国篮球职业联赛（CBA）上海大鲨鱼篮球俱乐部和美国国家篮球协会（NBA）休斯敦火箭，外号“小巨人”“移动长城”（The Walking Great Wall）。现任中国篮球协会主席。1998年4月，姚明入选王非执教的国家队，开始了职业篮球生涯。并在中国篮球协会（CBA）的上海大鲨鱼效力了五年
。现任中国篮球协会主席。1998年4月，姚明入选王非执教的国家队，开始了职业篮球生涯。并在中国篮球协会（CBA）的上海大鲨鱼效力了五年。2001夺得CBA常规赛最有价值球员及联赛最有价值球员[2]，2002年获得了CBA总冠军，但该

### prompt设置

In [16]:
prompt = "【指令】根据已知信息，简洁和专业的来回答问题。如果无法从中得到答案，请说 “根据已知信息无法回答该问题”，不允许在答案中添加编造成分，答案请使用中文。\n\
    \n【已知信息】{context}\n\n【问题】{question}\n"

In [19]:
query = "特朗普击败了谁"
retriever_docs = retriever.invoke(query)
prompt = ChatPromptTemplate.from_messages([prompt])
context = "\n\n".join([doc.page_content for doc in retriever_docs])
chain = prompt | llm
res = chain.invoke({"context": context, "question": query})
print(res.content)

特朗普在2016年11月8日当选为美国第45任总统，击败了民主党对手希拉里。
